In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import pandas as pd
import single_gen_func as sgf
from sdv.metadata import SingleTableMetadata
from sdv.evaluation.single_table import get_column_plot

********

# Базовая конфигурация

В данной секции необходимо определить базовую конфигурацию данных,
с которыми идет работа. Это включает в себя:
1. Создание списка из имен полей, содержащих значения типа дата. Если такие отсутствуют, можно пропустить данный пункт
2. Создание pandas DataFrame через вызов функции read_csv
3. Создания словаря, определяющего поля типа sdtype=id и их шаблоны регулярных выражений. При отсутствии шаблона передается None, в таком случае генерируется числовая или символьная инкрементальная последовательность. Подробнее обо всем в документации: https://docs.sdv.dev/sdv/reference/metadata-spec/sdtypes
4. Создание строки, содержащей название первичного ключа

In [ ]:
dates = ['scheduled_departure', 
         'scheduled_arrival', 
         'actual_departure', 
         'actual_arrival', 
         'constraint_date']

In [ ]:
df = sgf.read_csv('csvs/flights.csv', 5000, dates)
df.head(5)

In [ ]:
df.info()

In [ ]:
id_and_regex = {'flight_no': 'PG[0-9]{5}',
                'aircraft_code': None,
                'departure_airport': '[A-Z]{3}',
                'arrival_airport': '[A-Z]{3}'}

In [ ]:
pkey = 'flight_id'

***********

# Метаданные

В этом блоке:
1. Создается объект метаданных
2. Производится базовое автоматическое заполнение объекта на основе исходного DataFrame
3. Вызывается функция для основных дополнений объекта, а именно:
    а. Добавляется тип id и регулярки в соответствии с ранее заданными данными
    б. Добавляется тип datetime и формат для полей с датами при их наличии
    в. Добавляется первичный ключ
4. Кастомные апдейты метаданных по желанию

In [ ]:
metadata = SingleTableMetadata()

In [ ]:
metadata.detect_from_dataframe(data=df)

**При отсутствии дат в таблице функция вызывается с 3 аргументами**

In [ ]:
sgf.update_metadata(metadata, pkey, id_and_regex, dates)

In [ ]:
metadata

Образец для обновления метаданных хардкодом. Подробнее https://docs.sdv.dev/sdv/single-table-data/data-preparation/single-table-metadata-api

In [ ]:
metadata.update_column(
    column_name='aircraft_code',
    sdtype='id',
    regex_format='[0-9A-Z]{4}'
)

***

# Генерация

Далее находится код для генерации данных с использованием всех доступных моделей. 

В результате будет получен сгенерированный DataFrame, сам объект генератора (модели), а также объект отчета о качестве. Все это дополняется понятной визуализацией из основной функции. Аргументы задаются в соответствии с **документацией**, которую можно увидеть в **следующей ячейке**. Дополнительно можно построить график распределения числовых и категориальных полей, а также полей с датами

In [ ]:
help(sgf.generate_fake_data)

# GaussianCopula model
https://docs.sdv.dev/sdv/single-table-data/modeling/synthesizers/gaussiancopulasynthesizer

In [ ]:
copula_generator, copula_sample, copula_report = sgf.generate_fake_data('copula', metadata, df, len(df))

In [ ]:
copula_sample.head(5)

### Дополнительная визуализация распределения значений выбранного поля

In [ ]:
fig = get_column_plot(
    real_data=df,
    synthetic_data=copula_sample,
    column_name='scheduled_arrival',
    metadata=metadata
)   
fig.show()

# CTGAN model
https://docs.sdv.dev/sdv/single-table-data/modeling/synthesizers/ctgansynthesizer

In [ ]:
ctgan_generator, ctgan_sample, ctgan_report = sgf.generate_fake_data('ctgan', metadata, df, len(df))

In [ ]:
ctgan_sample.head(5)

In [ ]:
fig = get_column_plot(
    real_data=df,
    synthetic_data=ctgan_sample,
    column_name='status',
    metadata=metadata
)
    
fig.show()

# TVAE model
https://docs.sdv.dev/sdv/single-table-data/modeling/synthesizers/tvaesynthesizer

In [ ]:
tvae_generator, tvae_sample, tvae_report = sgf.generate_fake_data('tvae', metadata, df, len(df))

In [ ]:
tvae_sample.head(5)

In [ ]:
fig = get_column_plot(
    real_data=df,
    synthetic_data=tvae_sample,
    column_name='status',
    metadata=metadata
)
    
fig.show()

# CopulaGAN model
https://docs.sdv.dev/sdv/single-table-data/modeling/synthesizers/copulagansynthesizer

In [ ]:
copgan_generator, copgan_sample, copgan_report = sgf.generate_fake_data('copgan', metadata, df, len(df))

In [ ]:
copgan_sample.head(5)

In [ ]:
fig = get_column_plot(
    real_data=df,
    synthetic_data=copgan_sample,
    column_name='status',
    metadata=metadata
)

fig.show()